### Plan:
1. **Get tokens** for positive and negative tweets (by `token` in this context we mean `word`).
2. **Lemmatize** them (convert to base word forms). For that we will use a Part-of-Speech tagger.
3. **Clean'em up** (remove mentions, URLs, stop words).
4. **Prepare models** for the classifier, based on cleaned-up tokens.
5. **Run the Naive Bayes classifier**.

First, download necessary prepared samples.

In [1]:
import nltk

In [2]:
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

Get some sample positive/negative tweets.

In [3]:
from nltk.corpus import twitter_samples


We can either get the actual string content of those tweets:

In [4]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')

In [6]:
positive_tweets[50]

'@groovinshawn they are rechargeable and it normally comes with a charger when u buy it :)'

Or we can get a list of tokens using [tokenized method](https://www.nltk.org/howto/twitter.html) on `twitter_samples`.

In [7]:
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(tweet_tokens[50])

['@groovinshawn', 'they', 'are', 'rechargeable', 'and', 'it', 'normally', 'comes', 'with', 'a', 'charger', 'when', 'u', 'buy', 'it', ':)']


Now let's setup a Part-of-Speech tagger.  Download a perceptron tagger that will be used by the PoS tagger.

In [8]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Import Part-of-Speech tagger that will be used for lemmatization

In [9]:
from nltk.tag import pos_tag

Check how it works. Note that it returns tuples, where second element is a Part-of-Speech identifier.

In [10]:
pos_tag(tweet_tokens[50])

[('@groovinshawn', 'NN'),
 ('they', 'PRP'),
 ('are', 'VBP'),
 ('rechargeable', 'JJ'),
 ('and', 'CC'),
 ('it', 'PRP'),
 ('normally', 'RB'),
 ('comes', 'VBZ'),
 ('with', 'IN'),
 ('a', 'DT'),
 ('charger', 'NN'),
 ('when', 'WRB'),
 ('u', 'JJ'),
 ('buy', 'VB'),
 ('it', 'PRP'),
 (':)', 'JJ')]

In [11]:
tweet_tokens[50]

['@groovinshawn',
 'they',
 'are',
 'rechargeable',
 'and',
 'it',
 'normally',
 'comes',
 'with',
 'a',
 'charger',
 'when',
 'u',
 'buy',
 'it',
 ':)']

Let's write a function that will lemmatize twitter tokens.

For that, let's first fetch a WordNet resource. WordNet is a semantically-oriented dictionary of English - check chapter 2.5 of the NLTK book. In online version, this is part 5 [here](https://www.nltk.org/book/ch02.html).

In [12]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Now fetch PoS tokens so that they can be passed to `WordNetLemmatizer`.

In [13]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag

# Припустимо, що tweet_tokens вже визначено
tokens = tweet_tokens[50]

# Створюємо лематизатор
lemmatizer = WordNetLemmatizer()

# Функція для визначення частини мови для WordNet
def get_wordnet_pos(tag):
    if tag.startswith('NN'):  # Якщо іменник
        return 'n'
    elif tag.startswith('VB'):  # Якщо дієслово
        return 'v'
    else:
        return 'a'  # Якщо інше, приймаємо як прикметник

# Лематизуємо кожне слово
lemmatized_sentence = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tag(tokens)]
print(lemmatized_sentence)


['@groovinshawn', 'they', 'be', 'rechargeable', 'and', 'it', 'normally', 'come', 'with', 'a', 'charger', 'when', 'u', 'buy', 'it', ':)']


Note that it converts words to their base forms ('are' -> 'be', 'comes' -> 'come').

Now we can proceed to processing. 
During processing, we will perform cleanup:
  - remove URLs and mentions using regexes
  - after lemmatization, remove *stopwords*

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

What are these stopwords? Let's see some.

In [15]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

print(len(stop_words))

for i in range(10):
    print(stop_words[i])

179
i
me
my
myself
we
our
ours
ourselves
you
you're


Here comes the `process_tokens` function:

In [16]:
import re, string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag

def process_tokens(tweet_tokens):
    # Ініціалізуємо порожній список для очищених токенів
    cleaned_tokens = []
    
    # Завантажуємо стоп-слова для англійської мови
    stop_words = stopwords.words('english')
    
    # Створюємо лематизатор
    lemmatizer = WordNetLemmatizer()

    # Проходимо по кожному токену та його тегу
    for token, tag in pos_tag(tweet_tokens):
        # Регулярні вирази для виявлення URL та згадок користувачів
        if (re.search(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', token) or 
            re.search(r'(@[A-Za-z0-9_]+)', token)):
            continue  # Пропускаємо URL та згадки користувачів
        
        # Визначаємо частину мови для лематизації
        if tag.startswith('NN'):  # Якщо іменник
            pos = 'n'
        elif tag.startswith('VB'):  # Якщо дієслово
            pos = 'v'
        else:
            pos = 'a'  # Якщо інше, приймаємо як прикметник
   
        # Виконуємо лематизацію
        token = lemmatizer.lemmatize(token, pos)

        # Додаємо токен до очищених токенів, якщо він не є пунктуацією та не є стоп-словом
        if token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    
    # Повертаємо очищені токени
    return cleaned_tokens


Let's test `process_tokens`:

In [17]:
print("Before:", tweet_tokens[50])
print("After:", process_tokens(tweet_tokens[50]))

Before: ['@groovinshawn', 'they', 'are', 'rechargeable', 'and', 'it', 'normally', 'comes', 'with', 'a', 'charger', 'when', 'u', 'buy', 'it', ':)']
After: ['rechargeable', 'normally', 'come', 'charger', 'u', 'buy', ':)']


Run `process_tokens` on all positive/negative tokens.

In [20]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = [process_tokens(tokens) for tokens in positive_tweet_tokens]
negative_cleaned_tokens_list = [process_tokens(tokens) for tokens in negative_tweet_tokens]

Let's see how did the processing go.

In [21]:
print(positive_tweet_tokens[500])
print(positive_cleaned_tokens_list[500])

['Dang', 'that', 'is', 'some', 'rad', '@AbzuGame', '#fanart', '!', ':D', 'https://t.co/bI8k8tb9ht']
['dang', 'rad', '#fanart', ':d']


Let's see what is most common there. Add a helper function `get_all_words`:

In [22]:
def get_all_words(cleaned_tokens_list):
    return [w for tokens in cleaned_tokens_list for w in tokens]

all_pos_words = get_all_words(positive_cleaned_tokens_list)

Perform frequency analysis using `FreqDist`:

In [23]:
from nltk import FreqDist

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


Fine. Now we'll convert these to a data structure usable for NLTK's naive Bayes classifier ([docs here](https://www.nltk.org/_modules/nltk/classify/naivebayes.html)):

In [24]:
[tweet_tokens for tweet_tokens in positive_cleaned_tokens_list][0]

['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']

In [25]:
def get_token_dict(tokens):
    return dict([token, True] for token in tokens)
    
def get_tweets_for_model(cleaned_tokens_list):   
    return [get_token_dict(tweet_tokens) for tweet_tokens in cleaned_tokens_list]

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

Create two datasets for positive and negative tweets. Use 7000/3000 split for train and test data.

In [26]:
import random

# Створюємо навчальний датасет з позитивними та негативними твітами
positive_dataset = [(tweet_dict, "Positive") for tweet_dict in positive_tokens_for_model]
negative_dataset = [(tweet_dict, "Negative") for tweet_dict in negative_tokens_for_model]

# Об'єднуємо обидва датасети
dataset = positive_dataset + negative_dataset

# Випадково перемішуємо датасет
random.shuffle(dataset)

# Розділяємо датасет на навчальні та тестові дані
train_data = dataset[:7000]  # Перші 7000 твітів для навчання
test_data = dataset[7000:]  # Решта для тестування

# Перевірка результатів
print("Розмір навчального датасету:", len(train_data))
print("Розмір тестового датасету:", len(test_data))


Розмір навчального датасету: 7000
Розмір тестового датасету: 3000


Finally we use the nltk's NaiveBayesClassifier on the training data we've just created:

In [27]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

Accuracy is: 0.996
Most Informative Features
                      :( = True           Negati : Positi =   2079.2 : 1.0
                      :) = True           Positi : Negati =   1639.1 : 1.0
                     sad = True           Negati : Positi =     52.7 : 1.0
                     bam = True           Positi : Negati =     24.2 : 1.0
                follower = True           Positi : Negati =     20.1 : 1.0
                    glad = True           Positi : Negati =     19.5 : 1.0
                     x15 = True           Negati : Positi =     17.8 : 1.0
                  arrive = True           Positi : Negati =     17.3 : 1.0
                 forward = True           Positi : Negati =     15.6 : 1.0
              appreciate = True           Positi : Negati =     14.2 : 1.0
None


Note the Positive:Negative ratios.

Let's check some test phrase. First, download punkt sentence tokenizer ([docs here](https://www.nltk.org/api/nltk.tokenize.punkt.html))

In [25]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Now we won't rely on `twitter_samples.tokenized`, but rather will use a generic tokenization routine - `word_tokenize`.

In [28]:
from nltk.tokenize import word_tokenize

# Введений твіт
custom_tweet = "the service was #bad"

# Токенізуємо та очищуємо токени
custom_tokens = process_tokens(word_tokenize(custom_tweet))

# Класифікуємо твіт за допомогою класифікатора
# І передаємо токени у функцію get_token_dict(), яка приймає словник токенів та повертає словник ознак
# classifier.classify() використовує навчену модель для класифікації твіту
print(classifier.classify(get_token_dict(custom_tokens)))


Negative


Let's package it as a function:

In [30]:
def get_sentiment(text):
    # Очищення та токенізація тексту
    custom_tokens = process_tokens(word_tokenize(text))
    
    # Класифікація тексту за допомогою навченої моделі
    sentiment = classifier.classify(get_token_dict(custom_tokens))
    
    # Повертаємо результат класифікації
    return sentiment

# Вхідні тексти для аналізу
texts = ["bad", "service is bad", "service is really bad", "service is so terrible", "great service", "they stole my money", "#good"]

# Виведення результатів класифікації для кожного вхідного тексту
for t in texts:
    sentiment = get_sentiment(t)
    print(f"'{t}', Сентимент: '{sentiment}'")



'bad', Сентимент: 'Negative'
'service is bad', Сентимент: 'Negative'
'service is really bad', Сентимент: 'Negative'
'service is so terrible', Сентимент: 'Negative'
'great service', Сентимент: 'Positive'
'they stole my money', Сентимент: 'Negative'
'#good', Сентимент: 'Positive'


Seems ok!

# Homework 

In [35]:
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [36]:
# Імпорт модуля з відгуками на фільми
from nltk.corpus import movie_reviews

# Отримання списку файлових ідентифікаторів позитивних та негативних відгуків
positive_reviews = movie_reviews.fileids('pos')  # Список файлових ідентифікаторів позитивних відгуків
negative_reviews = movie_reviews.fileids('neg')  # Список файлових ідентифікаторів негативних відгуків

In [39]:
# Імпортуємо необхідний модуль
import nltk
# Імпортуємо корпус з відгуками на фільми
from nltk.corpus import movie_reviews

# Отримуємо список файлових ідентифікаторів позитивних та негативних відгуків
positive = movie_reviews.fileids('pos')
negative = movie_reviews.fileids('neg')

# Розділення відгуків на позитивні та негативні
positive_reviews = [nltk.corpus.movie_reviews.raw((positive[i])).replace("\n", "") for i in range(len(positive))]
negative_reviews = [nltk.corpus.movie_reviews.raw((negative[i])).replace("\n", "") for i in range(len(negative))]


In [40]:
# Імпорт необхідних модулів
from nltk.tokenize import word_tokenize

# Підготовка масивів з токенами для позитивних та негативних відгуків
positive_tokens_for_model = get_tweets_for_model([process_tokens(word_tokenize(positive_reviews[i])) for i in range(len(positive_reviews))])
negative_tokens_for_model = get_tweets_for_model([process_tokens(word_tokenize(negative_reviews[i])) for i in range(len(negative_reviews))])


In [42]:
# Створення позитивної та негативної вибірки для моделі
positive_dataset = [(token, "Positive") for token in positive_tokens_for_model]
negative_dataset = [(token, "Negative") for token in negative_tokens_for_model]


Now we won't rely on `twitter_samples.tokenized`, but rather will use a generic tokenization routine - `word_tokenize`.

In [43]:
# Об'єднання позитивного та негативного датасетів
test_dataset = positive_dataset + negative_dataset

# Випадкове перемішування датасету
random.shuffle(test_dataset)

In [45]:
# Визначення точності на вибірці
print("Accuracy is:", classify.accuracy(classifier, test_dataset))


Accuracy is: 0.596


### Conclusion: the provided code snippets demonstrate a structured approach to sentiment analysis, including data preprocessing, model training, evaluation, and accuracy assessment. These are fundamental steps in building effective sentiment analysis systems for various applications, including social media monitoring, customer feedback analysis, and opinion mining.

In [47]:
# Функція для обробки токенів тестових твітів
def process_tokens_test(tweet_tokens):
    cleaned_tokens = []  # Ініціалізація списку для очищених токенів
    stop_words = stopwords.words('english')  # Завантаження стоп-слів для англійської мови
    lemmatizer = WordNetLemmatizer()  # Ініціалізація лематизатора

    # Цикл обробки кожного токену та його тегу
    for token, tag in pos_tag(tweet_tokens):
        # Умова для відкидання URL, згадок користувачів та хештегів
        if (re.search(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', token) or 
            re.search(r'(@[A-Za-z0-9_]+)', token) or re.search(r'(#[A-Za-z0-9_]+)', token)):
            continue  # Пропускаємо токен, якщо він є URL, згадкою користувача або хештегом

        # Визначення частини мови для лематизації
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
   
        token = lemmatizer.lemmatize(token, pos)  # Лематизація токену

        # Додавання очищеного токену до списку, якщо він не є пунктуацією та не є стоп-словом
        if token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens  # Повертаємо список очищених токенів

# Отримання токенів позитивних та негативних твітів
positive_tweet_tokens_test = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens_test = twitter_samples.tokenized('negative_tweets.json')

# Очищення та обробка токенів позитивних та негативних твітів
positive_cleaned_tokens_list_test = [process_tokens_test(tokens) for tokens in positive_tweet_tokens_test]
negative_cleaned_tokens_list_test = [process_tokens_test(tokens) for tokens in negative_tweet_tokens_test]

# Підготовка токенів для моделі
positive_tokens_for_model_test = get_tweets_for_model(positive_cleaned_tokens_list_test)
negative_tokens_for_model_test = get_tweets_for_model(negative_cleaned_tokens_list_test)

In [48]:
# Створення позитивної та негативної вибірки для моделі
positive_dataset_test = [(tweet_dict, "Positive") for tweet_dict in positive_tokens_for_model_test]
negative_dataset_test = [(tweet_dict, "Negative") for tweet_dict in negative_tokens_for_model_test]

# Об'єднання позитивного та негативного датасетів
dataset_test = positive_dataset_test + negative_dataset_test

# Випадкове перемішування датасету
random.shuffle(dataset_test)

# Розділення датасету на навчальні та тестові дані
train_data_new = dataset_test[:7000]  # Перші 7000 твітів для навчання
test_data_new = dataset_test[7000:]  # Решта для тестування

In [49]:
# Створення нового класифікатора на основі навчальних даних
classifier_new = NaiveBayesClassifier.train(train_data_new)

# Ініціалізація списків для збереження точності з та без використання хештегів
acc_hashon = []
acc_hashoff = []

# Проведення порівняльного аналізу точності для 1500 ітерацій
for i in range(1500):
    # Випадкове перемішування датасету
    random.shuffle(dataset)
    
    # Виділення тестових даних
    test_data = dataset[7000:]
    
    # Оцінка точності з використанням класифікатора без урахування хештегів
    acc_hashon.append(classify.accuracy(classifier, test_data))
    
    # Оцінка точності з використанням класифікатора з урахуванням хештегів
    acc_hashoff.append(classify.accuracy(classifier_new, test_data))

# Обчислення середньої точності з та без хештегів
mean_accuracy_hashon = sum(acc_hashon)/len(acc_hashon)
mean_accuracy_hashoff = sum(acc_hashoff)/len(acc_hashoff)

# Виведення результатів
print("Середня точність з використанням хештегів:", mean_accuracy_hashon)
print("Середня точність без використання хештегів:", mean_accuracy_hashoff)

# Порівняння середньої точності з та без хештегів та виведення результату
if mean_accuracy_hashon > mean_accuracy_hashoff:
    print(f"Середня точність з використанням хештегів вище на: {round(100 * (mean_accuracy_hashon - mean_accuracy_hashoff), 4)} %")
else:
    print(f"Середня точність без використання хештегів вище на: {round(100 * (mean_accuracy_hashoff - mean_accuracy_hashon), 4)} %")


Середня точність з використанням хештегів: 0.9982095555555555
Середня точність без використання хештегів: 0.9989948888888889
Середня точність без використання хештегів вище на: 0.0785 %


### Conclusion: After conducting experiments with sentiment analysis models, it's evident that training a classifier on a dataset where hashtags are excluded leads to higher accuracy.

In [51]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

In [52]:
# Створення списку ознак та міток з датасету
features = [features for features, label in dataset]
labels = [label for features, label in dataset]

# Ініціалізація векторизатора
vectorizer = DictVectorizer()

# Ініціалізація класифікатора логістичної регресії
log_reg_cls = LogisticRegression()

# Створення конвеєру для векторизації та класифікації
pipeline = make_pipeline(vectorizer, log_reg_cls)

# Оцінка точності моделі за допомогою перехресної перевірки
accuracy_scores = cross_val_score(pipeline, features, labels, cv=5, scoring='accuracy')

# Виведення середньої точності
print("Mean accuracy:", accuracy_scores.mean())



Mean accuracy: 0.9963


In [53]:
# Порівняння середньої точності моделей Naive Bayes та Logistic Regression
if accuracy_scores.mean() < sum(acc_hashon)/len(acc_hashon):
    print(f"Середня точність моделі Naive Bayes вище, ніж середня точність моделі Logistic Regression.")
else:
    print(f"Середня точність моделі Logistic Regression вище, ніж середня точність моделі Naive Bayes.")


Середня точність моделі Naive Bayes вище, ніж середня точність моделі Logistic Regression.


### Conclusion: Based on the experiments conducted, it appears that the NaiveBayesClassifier outperforms the LogisticRegression model in this particular scenario.The Naive Bayes classifier achieved higher mean accuracy compared to Logistic Regression when evaluated on the dataset. This suggests that for sentiment analysis tasks with the provided dataset and features, the Naive Bayes approach may be more suitable and effective.